In [7]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder


# Load the dataset
df = pd.read_excel('PSP_Jan_Feb_2019.xlsx')

# Ensure all necessary columns are present in the DataFrame
required_columns = ['amount', '3D_secured', 'is_retry', 'hour', 'day_of_week', 'psp_success_rate', 'country', 'card', 'PSP']
missing_columns = [col for col in required_columns if col not in df.columns]
if missing_columns:
    print(f"Missing columns: {missing_columns}")
else:
    print("All required columns are present.")

# Define features and target variable
X = df[required_columns]
y = df['success']
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Feature scaling for Logistic Regression: Only scale numeric columns (amount, psp_success_rate, hour, day_of_week)
scaler = StandardScaler()
X_train[['amount', 'psp_success_rate', 'hour', 'day_of_week']] = scaler.fit_transform(X_train[['amount', 'psp_success_rate', 'hour', 'day_of_week']])
X_test[['amount', 'psp_success_rate', 'hour', 'day_of_week']] = scaler.transform(X_test[['amount', 'psp_success_rate', 'hour', 'day_of_week']])


# Baseline Model: Logistic Regression
log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train, y_train)
# Evaluate the Logistic Regression model
y_pred_log_reg = log_reg.predict(X_test)
print("Logistic Regression Model Performance:")
print(classification_report(y_test, y_pred_log_reg))
print(confusion_matrix(y_test, y_pred_log_reg))

# Advanced Model: Random Forest Classifier
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Evaluate the Random Forest model
y_pred_rf = rf_model.predict(X_test)
print("Random Forest Model Performance:")
print(classification_report(y_test, y_pred_rf))
print(confusion_matrix(y_test, y_pred_rf))

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Perform Grid Search with Cross-Validation
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Best hyperparameters from Grid Search
print(f"Best Parameters from Grid Search: {grid_search.best_params_}")

# Best model from Grid Search
best_rf_model = grid_search.best_estimator_

# Evaluate the best Random Forest model
y_pred_best_rf = best_rf_model.predict(X_test)
print("Best Random Forest Model Performance after Grid Search:")
print(classification_report(y_test, y_pred_best_rf))
print(confusion_matrix(y_test, y_pred_best_rf))

Missing columns: ['is_retry', 'hour', 'day_of_week', 'psp_success_rate']


KeyError: "['is_retry', 'hour', 'day_of_week', 'psp_success_rate'] not in index"